In [ ]:

import json
import sys
import os
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
     sys.path.insert(0, project_root)

import os
from google import genai
from google.genai import types
from src.llm.tools.PythonCalculatorTool import PythonCalculationTool

# Define the function declaration for the model

python_calculator_tool_local = PythonCalculationTool()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
python_calculator_decleration = python_calculator_tool_local.get_tool_declaration_data()

# Define the function to execute the Python code
execute_python_calculations = python_calculator_tool_local.execute_python_calculations
all_tools_functions = [
    execute_python_calculations,
]

# Define the function declaration for the model
all_tools_decleration = [
    types.Tool(function_declarations=[python_calculator_decleration])
]

config = {
    "tools": all_tools_decleration,
    "automatic_function_calling": {"disable": True},
    #"tool_config": {"function_calling_config": {"mode": "any"}},
}
# Configure the client
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))

chat = client.chats.create(model="gemini-2.0-flash", config=config)
response = chat.send_message("What is the sum of the first 50 prime numbers? Generate and run code for the calculation, and make sure you get all 50. Then divide by 0.04215215")
print(json.dumps(response.to_json_dict(), indent=4))

{
    "candidates": [
        {
            "content": {
                "parts": [
                    {
                        "function_call": {
                            "args": {
                                "python_code": "\nimport math\n\ndef is_prime(n):\n    if n <= 1:\n        return False\n    for i in range(2, int(math.sqrt(n)) + 1):\n        if n % i == 0:\n            return False\n    return True\n\nprimes = []\nnum = 2\nwhile len(primes) < 50:\n    if is_prime(num):\n        primes.append(num)\n    num += 1\n\nsum_of_primes = sum(primes)\ncalculation_result = str(sum_of_primes / 0.04215215)\n"
                            },
                            "name": "execute_python_calculations"
                        }
                    }
                ],
                "role": "model"
            },
            "citation_metadata": {
                "citations": [
                    {
                        "end_index": 261,
                        "start_index

In [7]:
def call_function(function_call, functions):
    function_name = function_call.name
    print(f"\nFunction Name: {function_name}\n")
    function_args = function_call.args
    print(f"\nFunction Args: {function_args}\n")
    for func in functions:
        if func.__name__ == function_name:
            return func(**function_args)

part = response.candidates[-1].content.parts[-1]

if part.function_call:
    result = call_function(part.function_call, all_tools_functions)

print(result)
    


Function Name: execute_python_calculations


Function Args: {'python_code': '\nimport math\n\ndef is_prime(n):\n    if n <= 1:\n        return False\n    for i in range(2, int(math.sqrt(n)) + 1):\n        if n % i == 0:\n            return False\n    return True\n\nprimes = []\nnum = 2\nwhile len(primes) < 50:\n    if is_prime(num):\n        primes.append(num)\n    num += 1\n\nsum_of_primes = sum(primes)\ncalculation_result = str(sum_of_primes / 0.04215215)\n'}

Calculation Result:
121393.57067195862
